In [173]:
import sys
sys.path.append('../src')

%load_ext autoreload
%autoreload 2

import numpy as np
from equation.burgers import Burgers1D, InitialCondition, BoundaryCondition
import torch
from equation.derivative import diff

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
eq = Burgers1D(nu=1)
ic = InitialCondition()
bc = BoundaryCondition()

In [159]:
input = {"interior": torch.ones(10, 2), "initial": torch.ones(10, 2)*2, "boundary": torch.ones(10, 2)}
input = {k: v.requires_grad_(True) for k, v in input.items()}

In [160]:
def func(input):
    return torch.sum(input**2, axis=-1)

In [161]:
u = {k: func(v) for k, v in input.items()}

In [162]:
u["interior"].grad_fn.next_functions

((<PowBackward0 at 0x7f9cdc347bb0>, 0),)

In [184]:
x = torch.tensor([1., 2. ,3.], requires_grad=True)
y = (x**2)[:, None]

In [181]:
diff(y, x)

tensor([2., 4., 6.], grad_fn=<MulBackward0>)

In [182]:
y

tensor([1., 4., 9.], grad_fn=<PowBackward0>)

In [185]:
torch.autograd.grad(y, x, create_graph=True, grad_outputs=torch.ones_like(y))

RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([3]) and output[0] has a shape of torch.Size([3, 1]).